In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import pickle
import os
from sklearn.model_selection import train_test_split

In [104]:
df=pd.read_excel("[로지스틱] 유의한 변수 추출(지체장애 이상지혈증).xlsx")
df.head()

,가구 월평균 지출액,월 혈압약 일수,운동 시간(분),성별,장애등급,가장 불편한부위,장애주된 원인,"외래진료 여부(치과,보건소,한의원)\n등 포함)",혈당치료 - 비약물요법,지난 2주간 아팠음 여부,...,16)만성질환명(골관절염(퇴행성_x000D_\n관절염)),17)만성질환명(류마티스 관절염),18)만성질환명(골다공증),19)만성질환명(척추측만증),20)만성질환명(허리목통증),21)만성질환명(피부염),22)만성질환명(백내장),23)만성질환명(우울증),24)만성질환명(암),25)만성질환명(기타)
0,250,0,90,1,6,36,64,2,0,1,...,0,0,0,0,0,0,0,0,0,0
1,150,30,0,1,6,14,69,1,0,1,...,0,0,0,0,0,0,0,0,0,1
2,200,0,60,2,5,38,48,1,0,1,...,0,0,0,0,1,0,0,0,0,1
3,154,30,60,2,5,38,48,2,0,1,...,0,0,0,0,1,0,0,1,0,0
4,270,0,0,2,3,36,48,1,2,1,...,1,0,0,0,0,0,0,0,0,0


In [105]:
df.columns

Index(['가구 월평균 지출액', '월 혈압약 일수', '운동 시간(분)', '성별', '장애등급', '가장 불편한부위',
       '장애주된 원인', '외래진료 여부(치과,보건소,한의원)\n등 포함)', '혈당치료 - 비약물요법',
       '지난 2주간 아팠음 여부', '최근 1년간 금연시도', '고른 식품 섭취여부', '본인 체형 평가',
       '건강관련서비스 이용여부', '이용_만성질환관리', '이용_건강검진', 'ADL_청소', 'ADL_빨래하기',
       '척추보조기-소지', '01)만성질환명(고혈압)', '02)만성질환명(뇌졸중,중풍)', '03)만성질환명(심근경색증)',
       '04)만성질환명(협심증)', '05)만성질환명(이상지혈증)', '06)만성질환명(당뇨병)', '07)만성질환명(갑상선장애)',
       '08)만성질환명(천식)', '09)만성질환명(폐결핵)', '10)만성질환명(폐질환(만성기관 지염,폐기종)',
       '11)만성질환명(위십이지장궤양)', '12)만성질환명(B형간염)', '13)만성질환명(C형간염)',
       '14)만성질환명(간경변증)', '15)만성질환명(신부전)', '16)만성질환명(골관절염(퇴행성_x000D_\n관절염))',
       '17)만성질환명(류마티스 관절염)', '18)만성질환명(골다공증)', '19)만성질환명(척추측만증)',
       '20)만성질환명(허리목통증)', '21)만성질환명(피부염)', '22)만성질환명(백내장)', '23)만성질환명(우울증)',
       '24)만성질환명(암)', '25)만성질환명(기타)'],
      dtype='object')

In [110]:
df1=df[['가구 월평균 지출액', '월 혈압약 일수', '운동 시간(분)', '성별', '장애등급', '가장 불편한부위',
       '장애주된 원인', '외래진료 여부(치과,보건소,한의원)\n등 포함)', '혈당치료 - 비약물요법',
       '지난 2주간 아팠음 여부', '최근 1년간 금연시도', '고른 식품 섭취여부', '본인 체형 평가',
       '건강관련서비스 이용여부', '이용_만성질환관리', '이용_건강검진', 'ADL_청소', 'ADL_빨래하기',
       '척추보조기-소지','05)만성질환명(이상지혈증)']]
df1 #2686건 

,가구 월평균 지출액,월 혈압약 일수,운동 시간(분),성별,장애등급,가장 불편한부위,장애주된 원인,"외래진료 여부(치과,보건소,한의원)\n등 포함)",혈당치료 - 비약물요법,지난 2주간 아팠음 여부,최근 1년간 금연시도,고른 식품 섭취여부,본인 체형 평가,건강관련서비스 이용여부,이용_만성질환관리,이용_건강검진,ADL_청소,ADL_빨래하기,척추보조기-소지,05)만성질환명(이상지혈증)
0,250,0,90,1,6,36,64,2,0,1,2,1,3,1,1,1,1,1,2,0
1,150,30,0,1,6,14,69,1,0,1,2,2,2,1,1,2,1,2,2,0
2,200,0,60,2,5,38,48,1,0,1,0,2,3,1,1,1,1,1,2,0
3,154,30,60,2,5,38,48,2,0,1,0,2,3,1,1,1,1,1,2,0
4,270,0,0,2,3,36,48,1,2,1,0,2,3,1,1,2,4,4,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,100,0,0,1,3,36,68,1,0,1,2,2,3,1,1,1,2,2,2,0
2682,100,0,60,1,5,38,69,2,0,2,2,1,4,1,1,2,1,1,2,0
2683,200,30,0,1,6,38,48,1,0,1,2,1,4,1,1,1,1,1,2,0
2684,200,0,0,2,6,31,48,2,0,1,2,1,3,1,1,1,1,1,2,1


### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [176]:
df1[df1['05)만성질환명(이상지혈증)']==1]

,가구 월평균 지출액,월 혈압약 일수,운동 시간(분),성별,장애등급,가장 불편한부위,장애주된 원인,"외래진료 여부(치과,보건소,한의원)\n등 포함)",혈당치료 - 비약물요법,지난 2주간 아팠음 여부,최근 1년간 금연시도,고른 식품 섭취여부,본인 체형 평가,건강관련서비스 이용여부,이용_만성질환관리,이용_건강검진,ADL_청소,ADL_빨래하기,척추보조기-소지,05)만성질환명(이상지혈증)
7,255,30,30,2,5,36,48,1,0,1,0,1,4,1,1,2,2,2,2,1
35,400,0,30,2,5,36,48,1,0,1,0,2,4,1,1,1,2,1,1,1
40,120,30,0,2,4,36,48,2,0,1,0,2,5,1,1,1,1,1,2,1
41,110,30,0,2,4,38,48,1,0,1,0,2,4,1,1,1,1,1,1,1
48,450,30,0,2,6,21,48,1,0,1,0,1,2,1,1,1,4,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,150,30,60,1,2,21,62,2,1,1,1,2,3,1,1,1,1,1,2,1
2673,170,30,0,2,5,38,48,1,0,1,0,2,4,1,1,1,1,1,2,1
2676,50,30,0,2,4,36,69,2,1,1,0,2,4,1,1,1,1,1,2,1
2677,250,0,0,2,4,36,48,2,1,1,0,2,5,1,1,1,1,1,2,1


In [177]:
out=df1.loc[[35]] # 이상지혈증 유 7, 
out

,가구 월평균 지출액,월 혈압약 일수,운동 시간(분),성별,장애등급,가장 불편한부위,장애주된 원인,"외래진료 여부(치과,보건소,한의원)\n등 포함)",혈당치료 - 비약물요법,지난 2주간 아팠음 여부,최근 1년간 금연시도,고른 식품 섭취여부,본인 체형 평가,건강관련서비스 이용여부,이용_만성질환관리,이용_건강검진,ADL_청소,ADL_빨래하기,척추보조기-소지,05)만성질환명(이상지혈증)
35,400,0,30,2,5,36,48,1,0,1,0,2,4,1,1,1,2,1,1,1


In [178]:
df2=df1.drop(out.index).reset_index(drop=True)
df2.head()

,가구 월평균 지출액,월 혈압약 일수,운동 시간(분),성별,장애등급,가장 불편한부위,장애주된 원인,"외래진료 여부(치과,보건소,한의원)\n등 포함)",혈당치료 - 비약물요법,지난 2주간 아팠음 여부,최근 1년간 금연시도,고른 식품 섭취여부,본인 체형 평가,건강관련서비스 이용여부,이용_만성질환관리,이용_건강검진,ADL_청소,ADL_빨래하기,척추보조기-소지,05)만성질환명(이상지혈증)
0,250,0,90,1,6,36,64,2,0,1,2,1,3,1,1,1,1,1,2,0
1,150,30,0,1,6,14,69,1,0,1,2,2,2,1,1,2,1,2,2,0
2,200,0,60,2,5,38,48,1,0,1,0,2,3,1,1,1,1,1,2,0
3,154,30,60,2,5,38,48,2,0,1,0,2,3,1,1,1,1,1,2,0
4,270,0,0,2,3,36,48,1,2,1,0,2,3,1,1,2,4,4,2,0


In [179]:
x_data=df2[['가구 월평균 지출액', '월 혈압약 일수', '운동 시간(분)', '성별', '장애등급', '가장 불편한부위',
       '장애주된 원인', '외래진료 여부(치과,보건소,한의원)\n등 포함)', '혈당치료 - 비약물요법',
       '지난 2주간 아팠음 여부', '최근 1년간 금연시도', '고른 식품 섭취여부', '본인 체형 평가',
       '건강관련서비스 이용여부', '이용_만성질환관리', '이용_건강검진', 'ADL_청소', 'ADL_빨래하기',
       '척추보조기-소지']]

In [180]:
y_data=df2[['05)만성질환명(이상지혈증)']]

In [181]:
y_data.value_counts()

05)만성질환명(이상지혈증)
0                  2131
1                   554
dtype: int64

---

In [182]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [183]:
df.columns

Index(['가구 월평균 지출액', '월 혈압약 일수', '운동 시간(분)', '성별', '장애등급', '가장 불편한부위',
       '장애주된 원인', '외래진료 여부(치과,보건소,한의원)\n등 포함)', '혈당치료 - 비약물요법',
       '지난 2주간 아팠음 여부', '최근 1년간 금연시도', '고른 식품 섭취여부', '본인 체형 평가',
       '건강관련서비스 이용여부', '이용_만성질환관리', '이용_건강검진', 'ADL_청소', 'ADL_빨래하기',
       '척추보조기-소지', '01)만성질환명(고혈압)', '02)만성질환명(뇌졸중,중풍)', '03)만성질환명(심근경색증)',
       '04)만성질환명(협심증)', '05)만성질환명(이상지혈증)', '06)만성질환명(당뇨병)', '07)만성질환명(갑상선장애)',
       '08)만성질환명(천식)', '09)만성질환명(폐결핵)', '10)만성질환명(폐질환(만성기관 지염,폐기종)',
       '11)만성질환명(위십이지장궤양)', '12)만성질환명(B형간염)', '13)만성질환명(C형간염)',
       '14)만성질환명(간경변증)', '15)만성질환명(신부전)', '16)만성질환명(골관절염(퇴행성_x000D_\n관절염))',
       '17)만성질환명(류마티스 관절염)', '18)만성질환명(골다공증)', '19)만성질환명(척추측만증)',
       '20)만성질환명(허리목통증)', '21)만성질환명(피부염)', '22)만성질환명(백내장)', '23)만성질환명(우울증)',
       '24)만성질환명(암)', '25)만성질환명(기타)'],
      dtype='object')

In [184]:
transformer = make_column_transformer(
        (OneHotEncoder(handle_unknown = "ignore"), ['성별','장애등급','가장 불편한부위', 
        '장애주된 원인', '외래진료 여부(치과,보건소,한의원)\n등 포함)', '혈당치료 - 비약물요법', '지난 2주간 아팠음 여부', '최근 1년간 금연시도',
       '고른 식품 섭취여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리', '이용_건강검진', 
                                                    'ADL_청소', 'ADL_빨래하기', '척추보조기-소지']),
        remainder='passthrough')
transformer = make_pipeline(transformer, MaxAbsScaler())
transformer.fit(x_train)
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

In [185]:
if not os.path.exists('models/disabled_chronic_disease/'):
    os.makedirs('models/disabled_chronic_disease/')

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(transformer, f)

In [186]:
#모델 생성

model = SVC(kernel='rbf', C=5.0, gamma=0.1, probability=True)#,class_weight=class_weight)

In [187]:
#모델 학습
model.fit(x_train, y_train)

#모델 검증
print('training data fit :', model.score(x_train, y_train)) 

print('test data fit :', model.score(x_test, y_test)) 

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(model, f)

C:\python\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training data fit : 0.9047365620010644
test data fit : 0.7878411910669976


In [188]:
x_test=out.drop(['05)만성질환명(이상지혈증)'],axis=1)
x_test

,가구 월평균 지출액,월 혈압약 일수,운동 시간(분),성별,장애등급,가장 불편한부위,장애주된 원인,"외래진료 여부(치과,보건소,한의원)\n등 포함)",혈당치료 - 비약물요법,지난 2주간 아팠음 여부,최근 1년간 금연시도,고른 식품 섭취여부,본인 체형 평가,건강관련서비스 이용여부,이용_만성질환관리,이용_건강검진,ADL_청소,ADL_빨래하기,척추보조기-소지
35,400,0,30,2,5,36,48,1,0,1,0,2,4,1,1,1,2,1,1


In [189]:
labels = ['무', '유']

In [190]:
x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)
label = labels[y_predict[0]]
y_predict = model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.7589081725018316


---

### 오버샘플링

In [191]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [192]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [193]:
df.columns

Index(['가구 월평균 지출액', '월 혈압약 일수', '운동 시간(분)', '성별', '장애등급', '가장 불편한부위',
       '장애주된 원인', '외래진료 여부(치과,보건소,한의원)\n등 포함)', '혈당치료 - 비약물요법',
       '지난 2주간 아팠음 여부', '최근 1년간 금연시도', '고른 식품 섭취여부', '본인 체형 평가',
       '건강관련서비스 이용여부', '이용_만성질환관리', '이용_건강검진', 'ADL_청소', 'ADL_빨래하기',
       '척추보조기-소지', '01)만성질환명(고혈압)', '02)만성질환명(뇌졸중,중풍)', '03)만성질환명(심근경색증)',
       '04)만성질환명(협심증)', '05)만성질환명(이상지혈증)', '06)만성질환명(당뇨병)', '07)만성질환명(갑상선장애)',
       '08)만성질환명(천식)', '09)만성질환명(폐결핵)', '10)만성질환명(폐질환(만성기관 지염,폐기종)',
       '11)만성질환명(위십이지장궤양)', '12)만성질환명(B형간염)', '13)만성질환명(C형간염)',
       '14)만성질환명(간경변증)', '15)만성질환명(신부전)', '16)만성질환명(골관절염(퇴행성_x000D_\n관절염))',
       '17)만성질환명(류마티스 관절염)', '18)만성질환명(골다공증)', '19)만성질환명(척추측만증)',
       '20)만성질환명(허리목통증)', '21)만성질환명(피부염)', '22)만성질환명(백내장)', '23)만성질환명(우울증)',
       '24)만성질환명(암)', '25)만성질환명(기타)'],
      dtype='object')

In [194]:
transformer = make_column_transformer(
        (OneHotEncoder(handle_unknown = "ignore"), ['성별','장애등급','가장 불편한부위', 
        '장애주된 원인', '외래진료 여부(치과,보건소,한의원)\n등 포함)', '혈당치료 - 비약물요법', '지난 2주간 아팠음 여부', '최근 1년간 금연시도',
       '고른 식품 섭취여부', '본인 체형 평가', '건강관련서비스 이용여부', '이용_만성질환관리', '이용_건강검진', 
                                                    'ADL_청소', 'ADL_빨래하기', '척추보조기-소지']),
        remainder='passthrough')
transformer = make_pipeline(transformer, MaxAbsScaler())
transformer.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

In [195]:
if not os.path.exists('models/disabled_chronic_disease/'):
    os.makedirs('models/disabled_chronic_disease/')

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(transformer, f)

In [196]:
#모델 생성

model = SVC(kernel='rbf', C=5.0, gamma=0.1, probability=True)#,class_weight=class_weight)

In [197]:
#모델 학습
model.fit(x_sm, y_sm)

#모델 검증
print('training data fit :', model.score(x_sm, y_sm)) 

print('test data fit :', model.score(x_test, y_test)) 

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(model, f)

C:\python\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training data fit : 0.9106783075889859
test data fit : 0.7196029776674938


In [198]:
x_test=out.drop(['05)만성질환명(이상지혈증)'],axis=1)
x_test

,가구 월평균 지출액,월 혈압약 일수,운동 시간(분),성별,장애등급,가장 불편한부위,장애주된 원인,"외래진료 여부(치과,보건소,한의원)\n등 포함)",혈당치료 - 비약물요법,지난 2주간 아팠음 여부,최근 1년간 금연시도,고른 식품 섭취여부,본인 체형 평가,건강관련서비스 이용여부,이용_만성질환관리,이용_건강검진,ADL_청소,ADL_빨래하기,척추보조기-소지
35,400,0,30,2,5,36,48,1,0,1,0,2,4,1,1,1,2,1,1


In [199]:
labels = ['무', '유']

In [200]:
x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)
label = labels[y_predict[0]]
y_predict = model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.58632671949231
